In [ ]:
!pip install scikit-learn 

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import pandas as pd

df = pd.read_csv("../data/raw/github_trending_repos.csv")

df.head()

In [ ]:
target = 'stars_period'
y = df[target]

relevant_features = ['stars', 'forks', 'contributors_count', 'search_language', 'timeframe', 'language']
dropped_features = ['owner', 'repo_name', 'description', 'url', 'scraped_at', 'stars_period']

0    1067
1     118
2     226
3     250
4      67
Name: stars_period, dtype: int64